# 导入包

In [1]:
from vina import Vina
from file_exits import ZincPdbqt, gz_writer
import os
from tqdm import tqdm
import pickle

print(os.getcwd())

/mnt/e/Python_Project/SMTarRNA/project/utils/Dock


# 创建Vina对象

In [17]:
v = Vina(sf_name='vina')
# 设置受体文件
v.set_receptor(rigid_pdbqt_filename='3a6p.pdbqt')
# 计算受体力场
# 3a6p:center=[13.492, 69.412, 28.593], box_size=[54, 92, 66]
# 3a6p:center=[13.492, 69.412, 28.593], box_size=[54, 92, 66]
v.compute_vina_maps(center=[13.492, 69.412, 28.593], box_size=[54, 92, 66])


mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -7.628          0          0
   2       -7.557      67.92      69.95
   3       -7.527      4.931      8.333
   4        -7.46      3.705      6.454
   5       -7.459      7.122      9.996
   6       -7.442      2.688      4.032
   7       -7.429      3.468      7.503
   8       -7.409      4.686      7.804
   9        -7.34      6.377      8.698
Computing Vina grid ... done.


# 准备配体

In [2]:
elements_list = ['C', 'H', 'O', 'N', 'S', 'P', 'BR', 'CL', 'F', 'I']
ligands_file_name = '../zinc/zinc_drug_like_3d_100k.pdbqt.gz'
# ligands_file_name = 'test_ligand.pdbqt.gz'
ligands = ZincPdbqt(ligands_file_name)
# iter_l = iter(ligands)
# len(ligands)
# next(iter_l)

In [29]:
e_zinc_list = ['ZINC000056316169', 'ZINC000276545805', 'ZINC000402873384', 'ZINC000884134733']
for id in e_zinc_list:
    ligand_str = ligands.data[id]
    v.set_ligand_from_string(ligand_str)
    # 对接
    v.dock(exhaustiveness=32)
    print(v.score()[0])



mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -7.511          0          0
Performing docking (random seed: -69854862) ... -8.448

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1        -8.52          0          0
   2       -8.333      6.906        9.4
   3       -8.269       1.93      2.718
   4       -8.258      45.34      50.13
   5       -8.149      17.17      19.83
   6       -8.124      7.583      11.26
   7       -8.112      15.45      17.84
   8        -8.07      3.556      8.011
   9       -8.055      10.25      13.05
  10       -7.995      7.491      10.38
  11       -7.946         10      12.69
  12       -7.905      30.54      34.48
  13       -7.892  

In [28]:
print(v.energies()[0, 0])
v.poses(n_poses=1)
v.score()

-7.511


array([-7.511, -9.486,  0.   ,  0.   ,  0.   , -0.718,  1.976, -0.718])

# 进行批量对接

In [ ]:
# v.set_ligand_from_file("test_ligand.pdbqt")
dock_results_file_name = ligands_file_name.replace('.pdbqt.gz', '_dock_results.pdbqt.gz')
dock_energy_dict_file_name = ligands_file_name.replace('.pdbqt.gz', '_dock_energy.pkl')
dock_energy = dict()
dock_results_file_name_list = [ligands_file_name.replace('.pdbqt.gz', f'_dock_results_{i}.pdbqt.gz') for i in range((len(ligands)//1000)+1)]
# print(dock_results_file_name_list)
# raise ValueError
writer = gz_writer(dock_results_file_name_list[0])
i = 0
f = open('docked_100k_zinc_id.txt', 'a+')
docked_ligand_id_tmp = list()
# 进行对接
for ligand_id, ligand_pdbqt in tqdm(ligands, desc='docking'):
    i += 1
    # 传入配体pdbqt字符串
    # print(ligand_id, ligand_pdbqt)
    docked_zinc_id = f.read()
    # 如果已对接，进行下一个
    if ligand_id in docked_zinc_id:
        continue
    # i += 1
    v.set_ligand_from_string(ligand_pdbqt)
    # 对接
    v.dock(exhaustiveness=32)
    # 将最后一个能量结果存入文件
    energy = v.energies()[0, 0]
    dock_energy[ligand_id] = energy
    # v.write_pose(dock_results_file_name)
    writer.writelines(v.poses(n_poses=1))
    docked_ligand_id_tmp.append(ligand_id+'\n')
    # if i % 100 == 0:
    #     writer.flush()
    if i % 1000 == 0:
        print('writing')
        writer.close()
        # 写入已对接分子号缓存
        f.writelines(docked_ligand_id_tmp)
        f.flush()
        # 清空暂存已对接分子列表
        docked_ligand_id_tmp = list()
        writer = gz_writer(dock_results_file_name_list[i//1000])
    # i+=1
f.writelines(docked_ligand_id_tmp)
f.close()
writer.close()
with open(dock_energy_dict_file_name, 'wb', buffering=0) as handle:
    pickle.dump(dock_energy, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [5]:
# print(v.score())
# print(v.optimize())
writer.close()

# 进行对接

In [8]:
v.dock(exhaustiveness=32)

Performing docking (random seed: -1274779443) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


# 输出文件

In [9]:
v.write_poses(pdbqt_filename="test_ligand_docking_results.pdbqt")


In [ ]:
v.poses()

In [5]:
ligands[:2]

[('ZINC000001768908',
  'REMARK  Name = ZINC000001768908\nREMARK                            x       y       z     vdW  Elec       q    Type\nREMARK                         _______ _______ _______ _____ _____    ______ ____\nROOT\nATOM      1  C   UNK    1       -0.699   1.479   1.216  0.00  0.00    +0.110 C \nENDROOT\nBRANCH   1   2\nATOM      2  N   UNK    1       -0.012   1.005   0.008  0.00  0.00    -0.620 N \nATOM      3  H   UNK    1       -0.500   1.339  -0.810  0.00  0.00    +0.440 HD\nATOM      4  H   UNK    1        0.935   1.354   0.000  0.00  0.00    +0.430 HD\nATOM      5  H   UNK    1        0.002  -0.004   0.002  0.00  0.00    +0.420 HD\nENDBRANCH   1   2\nBRANCH   1   6\nATOM      6  C   UNK    1       -0.720   2.985   1.224  0.00  0.00    +0.500 C \nATOM      7  O   UNK    1       -1.247   3.583   2.147  0.00  0.00    -0.630 OA\nATOM      8  O   UNK    1       -0.209   3.607   0.308  0.00  0.00    -0.670 OA\nENDBRANCH   1   6\nBRANCH   1   9\nATOM      9  C   UNK    1  